# Synthesize Detected Intensities
Compute synthesized intensity maps using our fake Hi-C instrument and our hydrodynamic simulations

In [ ]:
import time
import sys
import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
import traceback

import numpy as np
from scipy.interpolate import splprep,splev
import toolz
import h5py
import matplotlib
#matplotlib.use('cairo')
#print(matplotlib.get_backend())
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.animation
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import SkyCoord
from astropy.utils.console import ProgressBar
from astropy.visualization import AsinhStretch,SqrtStretch,ImageNormalize
import sunpy.sun.constants as sun_const
import sunpy.map
import sunpy
from scipy.interpolate import splrep,splev
import dask
import distributed
from dask_jobqueue import SLURMCluster
from IPython.display import HTML

import synthesizAR
from synthesizAR.observe import ObserverParallel
from synthesizAR.util import SpatialPair
from hi_c import InstrumentHiC,CustomEmissionModel

%matplotlib inline
#%load_ext snakeviz

Load in both the emission model and the active region

In [ ]:
active_region = synthesizAR.Field.restore('/scratch/wtb2/hi_c_simulation/field_checkpoint/')

In [ ]:
em_model = CustomEmissionModel.restore('/scratch/wtb2/hi_c_simulation/emission_model.json')

Need to modify observer a bit so that we avoid a bunch of unneeded interpolation.

In [ ]:
class Observer(ObserverParallel):
    
    def _interpolate_loops(self, ds):
        """
        Don't interpolate, just load them from the loops. The resolution is already sufficiently high.
        """
        # Interpolate all loops in HEEQ coordinates
        total_coordinates = []
        interpolated_loop_coordinates = []
        for loop in self.field.loops:
            interpolated_loop_coordinates.append(loop.field_aligned_coordinate.to(u.cm).value)
            total_coordinates.append(loop.coordinates.cartesian.xyz.value.T)

        total_coordinates = np.vstack(total_coordinates) * loop.coordinates.cartesian.xyz.unit

        return total_coordinates, interpolated_loop_coordinates
    
    def flatten_detector_counts(self, **kwargs):
        """
        Build custom Dask graph interpolating quantities for each in loop in time and space.
        """
        emission_model = kwargs.get('emission_model', None)
        interpolate_hydro_quantities = kwargs.get('interpolate_hydro_quantities', True)
        futures = {}
        for instr in self.instruments:
            futures[f'{instr.name}'] = instr.flatten_parallel(
                self.field.loops,self._interpolated_loop_coordinates, emission_model=emission_model)

        return futures

In [ ]:
hic = InstrumentHiC([1e4,2e4]*u.s,active_region.magnetogram.observer_coordinate,
                    fov={'min_x': -235*u.arcsec, 'max_x': -15*u.arcsec,
                         'min_y': 160*u.arcsec, 'max_y': 380*u.arcsec},
                    resolution=SpatialPair(x=0.6*u.arcsec/u.pixel, y=0.6*u.arcsec/u.pixel, z=None))

In [ ]:
obs = Observer(active_region,[hic],)

In [ ]:
#cluster = distributed.LocalCluster(threads_per_worker=1,n_workers=64,memory_limit='4GB')
cluster = SLURMCluster(queue='commons',
                       walltime='00:10:00',
                       local_directory='$SHARED_SCRATCH/wtb2',
                       memory='10GB',cores=2)

In [ ]:
print(cluster.job_script())

In [ ]:
cluster.start_workers(10)

In [ ]:
client = distributed.Client(cluster)
client

### Flatten Counts

In [ ]:
obs.build_detector_files('/scratch/wtb2/hi_c_simulation/',None)

In [ ]:
flatten_futures = obs.flatten_detector_counts(emission_model=em_model, interpolate_hydro_quantities=False)

### Bin Counts

In [ ]:
bin_futures = obs.bin_detector_counts('/storage-home/w/wtb2/data/hi_c_simulation/aia_res')

In [ ]:
bin_futures['Hi_C']['171'][0].exception()

## Animation

In [ ]:
m = sunpy.map.Map('/storage-home/w/wtb2/data/hi_c_simulation/Hi_C/171/map_t000000.fits')
#m = m.submap(SkyCoord(-210*u.arcsec,190.*u.arcsec,frame=m.coordinate_frame),
#             SkyCoord(-60.*u.arcsec,340*u.arcsec,frame=m.coordinate_frame))

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection=m)
m.plot(axes=ax,cmap=sunpy.cm.get_cmap('sdoaia171'),title=False,annotate=False,
       norm=ImageNormalize(vmin=0,vmax=1e1,stretch=AsinhStretch(0.05))#matplotlib.colors.SymLogNorm(1e-10,vmin=0.1,vmax=1e1)
      )
ax.grid(alpha=0)
fig.subplots_adjust(left=0,right=1,top=1,bottom=0,wspace=None,hspace=None)
#plt.savefig('hi_c_171_map.pdf',bbox_inches='tight')
lon,lat = ax.coords[0],ax.coords[1]
#lon.set_ticklabel_visible(False)
#lon.set_ticks_visible(False)
#lat.set_ticklabel_visible(False)
#lat.set_ticks_visible(False)
xpix,ypix = m.world_to_pixel(SkyCoord(Tx=-230*u.arcsec,Ty=165*u.arcsec,frame=m.coordinate_frame))
ax.text(int(xpix.value),int(ypix.value),
        f'$t={hic.observing_time[0].value:.0f}$ {hic.observing_time.unit}',
        fontsize=20,color='w',)

In [ ]:
fig = plt.figure(figsize=(10,10))
m = sunpy.map.Map('/storage-home/w/wtb2/data/hi_c_simulation/aia_res/Hi_C/171/map_t000000.fits')
m = m.submap(SkyCoord(-210*u.arcsec,190.*u.arcsec,frame=m.coordinate_frame),
             SkyCoord(-60.*u.arcsec,340*u.arcsec,frame=m.coordinate_frame))
ax = fig.gca(projection=m)
im = m.plot(axes=ax,cmap=sunpy.cm.get_cmap('sdoaia171'),
            norm=ImageNormalize(vmin=0,vmax=1e3,stretch=AsinhStretch(0.05)),
            annotate=False,
            title=False)
ax.grid(alpha=0)
lon,lat = ax.coords[0],ax.coords[1]
lon.set_ticklabel_visible(False)
lon.set_ticks_visible(False)
lat.set_ticklabel_visible(False)
lat.set_ticks_visible(False)
#plt.tight_layout()
fig.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=None,wspace=None)
xpix,ypix = m.world_to_pixel(SkyCoord(Tx=-205*u.arcsec,Ty=195*u.arcsec,frame=m.coordinate_frame))
xpix,ypix = int(xpix.value),int(ypix.value)
text = ax.text(xpix,ypix,
               f'$t={hic.observing_time[0].value:.0f}$ {hic.observing_time}',
               fontsize=20,color='w',)
def update(i):
    m = sunpy.map.Map(f'/storage-home/w/wtb2/data/hi_c_simulation/aia_res/Hi_C/171/map_t{i:06d}.fits')
    m = m.submap(SkyCoord(-210*u.arcsec,190.*u.arcsec,frame=m.coordinate_frame),
                 SkyCoord(-60.*u.arcsec,340*u.arcsec,frame=m.coordinate_frame))
    im.set_data(m.data)
    #ax.set_title(f'$t={hic.observing_time[i]:.0f}$',fontsize=20)
    text.set_text(f'$t={hic.observing_time[i].value:.0f}$ {hic.observing_time.unit}')
    return im,text
anim = matplotlib.animation.FuncAnimation(fig,update,frames=hic.observing_time.shape[0],
                                          blit=True, repeat=True, interval=20)

In [ ]:
dpi = int(np.sqrt(m.data.shape[0]*m.data.shape[1] / (fig.get_figheight()*fig.get_figwidth()))*4)

In [ ]:
anim.save('hi_c_movie_aia_res.mp4',writer='ffmpeg',dpi=dpi,)

In [ ]:
1667/1000 * 20

In [ ]:
m.data.shape